In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.alemutdf import get_all_sample_mut_df, get_multi_exp_max_freq_mut_df
from util.mut import is_coding_mut
from util.metadata import get_condition_val_dict, get_condition_field_val_set
from util.genome import get_K12_pos_from_BOP27

In [2]:
muts_df = get_all_sample_mut_df("./data/muts/published/")
# unpublished_muts_df = get_all_sample_mut_df("./data/muts/unpublished/")  # I've currently placed GYD into the public mutation set.
# muts_df = muts_df.append(unpublished_muts_df)
display(set(muts_df["exp"]))

  0%|          | 0/20 [00:00<?, ?it/s]

./data/muts/published//TEE Ara.csv
./data/muts/published//42C.csv
./data/muts/published//Iron-ALE_mut.csv
./data/muts/published//MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2_mut.csv
./data/muts/published//TOL adipic acid_mut.csv


  0%|          | 0/40 [00:00<?, ?it/s]

./data/muts/published//TOL n-butanol_mut.csv
./data/muts/published//Epistasis2011.csv
./data/muts/published//PGI.csv
./data/muts/published//Auxotrophic strains hisD gltA.csv


  0%|          | 0/24 [00:00<?, ?it/s]

...

 32%|███▏      | 21/66 [00:00<00:00, 208.12it/s]

./data/muts/published//pgi_mut.csv


100%|██████████| 10/10 [00:00<00:00, 207.98it/s]

./data/muts/published//C13.csv
./data/muts/published//gnd_mut.csv


{'42C',
 '42C_Tenaillon',
 'Auxotrophic_strains_hisD_gltA',
 'Auxotrophic_strains_hisD_gltB',
 'Auxotrophic_strains_hisD_pyrC',
 'C13',
 'C321',
 'C321.∆A',
 'C321.∆A.earlyfix',
 'ECNR2.1',
 'Epistasis2011',
 'GLU',
 'GYD',
 'Iron-ALE',
 'LTEE',
 'LTEE_Ara',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-m-tartrate2',
 'MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2',
 'PGI',
 'SER',
 'SSW_AC',
 'SSW_GLU_AC',
 'SSW_GLU_GLY',
 'SSW_GLU_XYL',
 'SSW_GLY',
 'SSW_XYL',
 'TEE',
 'TEE_Ara',
 'TOL_2,3-butanediol',
 'TOL_adipic_acid',
 'TOL_coumaric_acid',
 'TOL_glutaric_acid',
 'TOL_hexamethylenediamine',
 'TOL_hexanoic_acid',
 'TOL_isobutyric_acid',
 'TOL_n-butanol',
 'TOL_octanoic_acid',
 'TOL_propanediol',
 'TOL_putrescine',
 'evo',
 'gnd',
 'pgi',
 'pts',
 'sdh',
 'tpi'}

In [3]:
l = [
    "42c_tenaillon",
    "ltee",
    "ltee_ara",
    "tee",
    "tee_ara",
    "tm_glugly",
    "epistasis2011",
    'C321',
    'C321.∆A',
    'C321.∆A.earlyfix',
    'ECNR2.1',
    'Auxotrophic_strains_hisD_gltA',
    'Auxotrophic_strains_hisD_gltB',
    'Auxotrophic_strains_hisD_pyrC',
]
exp_to_remove_l = [x.lower() for x in l]
muts_df = muts_df[~muts_df.exp.str.lower().isin(exp_to_remove_l)]
set(muts_df.exp)

{'42C',
 'C13',
 'GLU',
 'GYD',
 'Iron-ALE',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-m-tartrate2',
 'MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2',
 'PGI',
 'SER',
 'SSW_AC',
 'SSW_GLU_AC',
 'SSW_GLU_GLY',
 'SSW_GLU_XYL',
 'SSW_GLY',
 'SSW_XYL',
 'TOL_2,3-butanediol',
 'TOL_adipic_acid',
 'TOL_coumaric_acid',
 'TOL_glutaric_acid',
 'TOL_hexamethylenediamine',
 'TOL_hexanoic_acid',
 'TOL_isobutyric_acid',
 'TOL_n-butanol',
 'TOL_octanoic_acid',
 'TOL_propanediol',
 'TOL_putrescine',
 'evo',
 'gnd',
 'pgi',
 'pts',
 'sdh',
 'tpi'}

## remove problem and starting strain mutations

In [4]:
# Different experiments have different strings for position (some with commas, some without), therefore going ahead and changing them all to integers
muts_df.Position = muts_df.Position.apply(lambda x: int(str(x).replace(",","")))
muts_df.Position = muts_df.Position.astype(int)

In [5]:
muts_df = muts_df[~(
# For whatever reason, the gene annotations can change between version of breseq or runs
#     ((muts_df.Gene == "gatC")
#      | (muts_df.Gene == "gatC, gatC"))
    ((muts_df.Position == 2173361)
       | (muts_df.Position == 2173363)
       | (muts_df.Position == 2173364))
    & ((muts_df["Sequence Change"] == "Δ2 bp")
       | (muts_df["Sequence Change"] == "Δ1 bp"))
)]

muts_df = muts_df[~(
#     (muts_df.Gene == "[crl]")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 257908)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "insB1, insA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1978503)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "ychE, oppA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1299499)
    & (muts_df["Sequence Change"] == "Δ1,199 bp"))]

muts_df = muts_df[~(
# For whatever reason, the gene annotations can change between version of breseq or runs
#     ((muts_df.Gene == "glpR")
#      | (muts_df.Gene == "glpR, glpR")
#      | (muts_df.Gene == "glpR/glpR"))
    (muts_df.Position == 3560455)
    & (muts_df["Sequence Change"] == "+G"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 4296381)
    & (muts_df["Sequence Change"] == "+GC"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 4296380)
    & (muts_df["Sequence Change"] == "+CG"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "insB1,insA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1978503)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     ((muts_df.Gene == "insA/uspC") | (muts_df.Gene == "insA, uspC"))  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1979486)
    & (muts_df["Sequence Change"] == "IS5 (+) +4 bp"))]


muts_df = muts_df[~(
#     (muts_df.Gene == "corA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.exp != "GLU")  # This is the ALE that originated the corA mutation that is found in all CCK samples, therefore should track
    & (muts_df.Position == 4001645)
    & (muts_df["Sequence Change"] == "Δ5 bp"))]


muts_df = muts_df[~(
#     (muts_df.Gene == "wcaA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.exp != "GLU")  # This is the ALE that originated the corA mutation that is found in all CCK samples, therefore should track
    & (muts_df.Position == 2132787)
    & (muts_df["Details"] == "I204S (ATC→AGC)"))]


p = {4293212,
 4293403,
 4293430,
 4296060,
 4296154,
 4296189,
 4296190,
 4296191,
 4296267,
 4296268,
 4296286,
 4296380,
 4296382}
ch = {'+C', '+G', 'A→C', 'A→G', 'A→T', 'C→A', 'C→T', 'T→C'}
muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")
    (muts_df["Position"].isin(p))
    & (muts_df["Sequence Change"].isin(ch))
       )]

display(len(muts_df), muts_df.head())

9898

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID
0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN
2,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN
6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,"[rph], [rph]",NaN,NaN
7,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN
8,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN


## Remove experiment specific starting strain mutations

In [7]:
# Filter the following mutations out if they come from specific experiments.
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="cyoB") & (muts_df["Details"]=="W190* (TGG→TAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="eno") & (muts_df["Details"]=="L193Q (CTG→CAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="chbF") & (muts_df["Details"]=="V133E (GTA→GAA)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


cck_exp_list = [
    "Central_carbon_knockout_tpiA",
    "Central_carbon_knockout_PTS",
    "Central_carbon_knockout_pgi",
    "Central_carbon_knockout_sdh",
    "Central_carbon_knockout_gnd",
    "Central_carbon_knockout_glucose_evolution"
]

muts_df = muts_df[~(
    (muts_df.exp.isin(cck_exp_list))
    & (
        (muts_df["Gene"]=="wcaA")
        | (muts_df["Gene"]=="corA")
    )
)]  # The details of the mutations to filter out should probably include the sequence change.

In [8]:
# Since the majority of our comparisons have to do with endpoints, just look at endpoints.
# If not limiting to endpoint, the mutations used in associated mutations to conditions will need filtering.
# This occurs in a later NB of the pipeline.
muts_df = get_multi_exp_max_freq_mut_df(muts_df, endpoint_flask_only=False)
muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID
0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN
2,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN
6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,"[rph], [rph]",NaN,NaN
7,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN
8,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN


In [9]:
# ensure all mutations are using NC_000913 positions.
genome_pos_transform_exp_l = ["ERS",
                              'Central_carbon_knockout_PTS',
                              'Central_carbon_knockout_glucose_evolution',
                              'Central_carbon_knockout_gnd',
                              'Central_carbon_knockout_pgi',
                              'Central_carbon_knockout_sdh',
                              'Central_carbon_knockout_tpiA',]
muts_df.Position = muts_df.apply(lambda row: get_K12_pos_from_BOP27(row.Position) if row.exp in genome_pos_transform_exp_l else row.Position, axis=1)

In [10]:
muts_df["coding"] = muts_df.Details.apply(is_coding_mut)
muts_df["Gene"] = muts_df.apply(lambda row: row["Gene"].replace(", ","/") if not row["coding"] else row["Gene"], axis=1)

muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID,coding
0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN,True
2,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN,True
6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,"[rph], [rph]",NaN,NaN,True
7,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN,True
8,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN,True


In [11]:
print(len(muts_df))
muts_df = muts_df[muts_df.presence >= 0.5]
print(len(muts_df))

6085
5773


In [12]:
# This work is also currently duplicated NB4 
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph], [rph]" else a)
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph],[rph]" else a)
muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID,coding
0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN,True
2,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN,True
6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,rph,NaN,NaN,True
7,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN,True
8,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN,True


In [13]:
# removing unnecessary column
muts_df = muts_df.drop(columns=["Mut ID"])

In [14]:
# Amount of ALEs before removal
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

# removing hypermutators since they will affect associations

muts_df = muts_df[
    ~(
        (muts_df.exp == "GLU")
        & ((muts_df.ale == 5) | (muts_df.ale == 7))
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_2,3-butanediol")
        & (muts_df.ale == 3)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_adipic_acid")
        & (muts_df.ale == 5)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_hexamethylenediamine")
        & (muts_df.ale == 6)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_octanoic_acid")
        & (muts_df.ale == 5)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_propanediol")
        & ((muts_df.ale == 1) | (muts_df.ale == 2) | (muts_df.ale == 3) | (muts_df.ale == 4) | (muts_df.ale == 5) | (muts_df.ale == 7) | (muts_df.ale == 8))
    )
]

print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

223
210


In [15]:
# removing samples not incuded in GYD MS for AVA MS

muts_df = muts_df[
    ~(
        (muts_df.exp == "GYD")
        & ((muts_df.ale == 35) | (muts_df.ale == 34))
    )
]
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

208


In [16]:
muts_df.to_pickle("./data/1_df.pkl")

In [17]:
display(len(muts_df.exp.unique()))

35

In [18]:
for e in muts_df.exp.unique():
    print(e)

42C
C13
GLU
GYD
Iron-ALE
MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2
MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2
MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2
MG1655-M9-NC_000913_3gb-stationary-37-m-tartrate2
MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2
PGI
SER
SSW_AC
SSW_GLU_AC
SSW_GLU_GLY
SSW_GLU_XYL
SSW_GLY
SSW_XYL
TOL_2,3-butanediol
TOL_adipic_acid
TOL_coumaric_acid
TOL_glutaric_acid
TOL_hexamethylenediamine
TOL_hexanoic_acid
TOL_isobutyric_acid
TOL_n-butanol
TOL_octanoic_acid
TOL_propanediol
TOL_putrescine
evo
gnd
pgi
pts
sdh
tpi


In [19]:
muts_df["exp ALE"] = muts_df.apply(lambda r: r.exp + ' ' + str(r.ale), axis=1)
display(len(muts_df["exp ALE"].unique()), muts_df["exp ALE"].unique())

208

array(['42C 1', '42C 2', '42C 3', '42C 4', '42C 5', '42C 6', '42C 7',
       '42C 8', '42C 9', '42C 10', 'C13 1', 'C13 2', 'C13 3', 'C13 4',
       'C13 5', 'C13 6', 'GLU 3', 'GLU 4', 'GLU 6', 'GLU 8', 'GLU 9',
       'GLU 10', 'GYD 1', 'GYD 2', 'GYD 3', 'GYD 4', 'GYD 5', 'GYD 6',
       'GYD 7', 'GYD 8', 'GYD 9', 'GYD 10', 'GYD 11', 'GYD 12', 'GYD 13',
       'GYD 14', 'GYD 15', 'GYD 16', 'GYD 17', 'GYD 18', 'GYD 19',
       'GYD 20', 'GYD 21', 'GYD 22', 'GYD 23', 'GYD 24', 'GYD 25',
       'GYD 26', 'GYD 27', 'GYD 28', 'GYD 29', 'GYD 30', 'Iron-ALE 17',
       'Iron-ALE 18', 'Iron-ALE 19', 'Iron-ALE 20',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2 1',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2 1',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2 2',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2 1',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2 2',
       'MG1655-M9-NC_000913_3gb-stationary-37-m-tartrate2 1',
       'MG1

In [20]:
len(muts_df)

2318